In [5]:
import boto3
from botocore.exceptions import ClientError
import os
import traceback

access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
session_token = os.environ.get('AWS_SESSION_TOKEN')
#region = os.environ.get('AWS_DEFAULT_REGION')
region = 'us-east-1'

### 任务脚本示例：对现有的资源tag进行规范化
def processTags(aws_access_key_id, aws_secret_access_key, aws_session_token, region, plan=True):
    session = boto3.Session(
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_session_token=aws_session_token,
        region_name=region
    ) 

    # 创建一个名为tag_api的ResourceGroupsTaggingAPI客户端
    tag_api = session.client("resourcegroupstaggingapi")

    # 获取全部资源
    paginator = tag_api.get_paginator("get_resources")

    # 遍历资源并查找包含“项目”标签的资源
    for page in paginator.paginate():
        for resource in page['ResourceTagMappingList']:
            resource_arn = resource['ResourceARN']
            strarn=f"Resource ARN: {resource_arn}"
            #print(resource_arn)
            
            project_value = ""
            department_finance_project_exist = False

            for tag in resource['Tags']:
                key_no_space_lower = tag['Key'].replace(" ", "").lower()
                if key_no_space_lower == "project":
                    project_value = tag['Value']
                    stroriginal=f"Original Project Tag: {tag['Key']}={tag['Value']}"
                elif key_no_space_lower == "department:finance:project":
                    department_finance_project_exist = True

            if project_value and not department_finance_project_exist:
                resource_type, resource_id = get_resource_info(resource_arn)
                if plan:
                    print(f"Plan to add tag: department:finance:project={project_value}\n")
                else:
                    add_new_tag(tag_api, resource_arn, "department:finance:project", project_value)
                    resource_type, resource_id = get_resource_info(resource_arn)
                    print(f"Added 'department:finance:project' tag with value: {project_value} for Resource Type: {resource_type} and Resource Identifier: {resource_id}\n")

In [ ]:
### 跳过指定的account
skipaccounts = ['111111111111']

#Assume Role to manage account
sts_client = boto3.client('sts', aws_access_key_id=access_key, aws_secret_access_key=secret_key, aws_session_token=session_token, region_name=region)

roleARN = 'arn:aws:sts::000000000000:role/admin-role'
role_session_name = 'sname-automationadmin-getorgaccounts'
assumedRoleObject = sts_client.assume_role(RoleArn=roleARN, RoleSessionName=role_session_name)

assumed_access_key = assumedRoleObject['Credentials']['AccessKeyId']
assumed_secret_key = assumedRoleObject['Credentials']['SecretAccessKey']
assumed_session_token = assumedRoleObject['Credentials']['SessionToken']
org_client = boto3.client('organizations', aws_access_key_id=assumed_access_key, aws_secret_access_key=assumed_secret_key, aws_session_token=assumed_session_token, region_name=region)

account_list = []
token = None
while True:
    if token:
        response = org_client.list_accounts(NextToken=token)
    else:
        response = org_client.list_accounts()
    account_list += response['Accounts']
    if 'NextToken' in response:
        token = response['NextToken']
    else:
        break

for acct in account_list:    
    if acct['Id'] in skipaccounts:
        continue
    print("\n\nProcessing AccountID %s" % acct['Id'])
    
    try:
        roleARN = 'arn:aws:iam::' + acct['Id'] + ':role/' + 'automation-role'
        role_session_name = 'sname-automation-acct'
        assumedRoleObject = sts_client.assume_role(RoleArn=roleARN, RoleSessionName=role_session_name)
    
        assumed_access_key = assumedRoleObject['Credentials']['AccessKeyId']
        assumed_secret_key = assumedRoleObject['Credentials']['SecretAccessKey']
        assumed_session_token = assumedRoleObject['Credentials']['SessionToken']

        ## Execute a python program to operate on AWS resources within the specified account
        #example: taskfunction(assumed_access_key, assumed_secret_key, assumed_session_token, region, False)
        processTags(assumed_access_key, assumed_secret_key, assumed_session_token, region, False)
    except Exception as e:
        tb = traceback.extract_tb(e.__traceback__)[-1]  # 获取异常的traceback对象，提取最后一个堆栈信息
        lineno = tb.lineno  # 通过 tb 对象获取异常行数
        print(f"发生了一个异常: {e}, 出现在文件 {tb.filename} 的行 {lineno}")